In [120]:
import numpy as np

In [121]:
cars = ['1', '3', '4', '6', '7', '12', '14', '15', '17', '18', 
        '19', '20', '21', '22', '23', '24', '25', '26', '27', 
        '28', '29', '59', '60', '64', '66', '88', '98']
detectors = ['numenta', 'random', 'bayes', 'expose']
det_names = ['Numenta HTM', 'Random Cut Forest', 'Bayesian Changepoint', 'EXPoSE']
thresholds = ['0.36', '0.997', '0.39', '0.992']

In [122]:
tol = 5000

all_accs = []
all_negs = []
for car in cars:
    labels = np.loadtxt('labels/indy2018-' + car + '-labels.txt')
    events = np.array(list(set(labels.flatten())))
    
    accs = []
    negs = []
    for i in range(len(detectors)):
        total = np.loadtxt('results/' + detectors[i] + 
                           '/anomaly_' + thresholds[i] + 
                           '_indy2018-' + car + '-vspeed.csv')
        maskt = total[:, 2] == 1
        anomaly = total[maskt]
        speeds = anomaly[:, 0]
        mask1 = speeds >= labels[0, 0] - tol
        mask2 = speeds <= labels[-1, 1] + tol
        masks = mask1 & mask2
        middle = speeds[masks]
        
        k = 0
        matches = [0] * labels.shape[0]
        for j in range(middle.shape[0]):
            if middle[j] < labels[k, 0] - tol:
                continue
            elif middle[j] >= labels[k, 0] - tol and \
                    middle[j] <= labels[k, 1] + tol:
                matches[k] += 1
            elif middle[j] > labels[k, 1] + tol:
                k += 1
                j -= 1
            else:
                print('in else')
            if k == labels.shape[0]:
                break
        
        match = sum([1 for cnt in matches if cnt > 0])
        miss = middle.shape[0] - sum(matches)
        accuracy = match / labels.shape[0]
        negative = miss / middle.shape[0]
        
        accs.append(accuracy)
        negs.append(negative)
        
    all_accs.append(accs)
    all_negs.append(negs)

In [123]:
len(all_accs), len(all_accs[0])

(27, 4)

In [124]:
print('recall rate: ')
print('car, {:s}, {:s}, {:s}, {:s}'.format(\
        det_names[0], det_names[1], det_names[2], det_names[3]))
for i in range(len(all_accs)):
    print('{:3s}, {:6.2f}%, {:6.2f}%, {:6.2f}%, {:6.2f}%'\
          .format(cars[i], all_accs[i][0] * 100, all_accs[i][1] * 100, \
                  all_accs[i][2] * 100, all_accs[i][3] * 100))

recall rate: 
car, Numenta HTM, Random Cut Forest, Bayesian Changepoint, EXPoSE
1  ,  81.25%,  50.00%,  93.75%,  81.25%
3  ,  80.00%,  60.00%, 100.00%,  90.00%
4  ,  58.33%,  58.33%,  75.00%,  66.67%
6  ,  81.82%,  63.64%,  90.91%,  90.91%
7  ,  80.00%,  40.00%,  80.00%,  73.33%
12 ,  84.62%,  61.54%,  76.92%,  76.92%
14 ,  73.33%,  66.67%,  66.67%,  60.00%
15 ,  77.78%,  55.56%,  88.89%,  66.67%
17 ,  70.00%,  50.00%,  80.00%,  70.00%
18 ,  66.67%,  55.56%,  77.78%,  88.89%
19 ,  78.57%,  64.29%, 100.00%,  57.14%
20 ,  69.23%,  53.85%, 100.00%,  61.54%
21 ,  66.67%,  46.67%,  80.00%,  73.33%
22 ,  83.33%,  58.33%,  91.67%,  91.67%
23 ,  71.43%,  64.29%,  85.71%,  92.86%
24 ,  50.00%,  41.67%,  83.33%,  50.00%
25 ,  83.33%,  25.00%,  83.33%,  91.67%
26 ,  64.71%,  64.71%,  88.24%,  70.59%
27 ,  60.00%,  60.00%,  80.00%,  80.00%
28 ,  80.00%,  40.00%,  80.00%,  90.00%
29 ,  60.00%,  60.00%,  90.00%,  90.00%
59 ,  66.67%,  55.56%, 100.00%,  88.89%
60 ,  60.00%,  90.00%, 100.00%,  80.00%


In [125]:
print('false positive: ')
print('car, {:s}, {:s}, {:s}, {:s}'.format(\
        det_names[0], det_names[1], det_names[2], det_names[3]))
for i in range(len(all_accs)):
    print('{:3s}, {:6.2f}%, {:6.2f}%, {:6.2f}%, {:6.2f}%'\
          .format(cars[i], all_negs[i][0] * 100, all_negs[i][1] * 100, \
                  all_negs[i][2] * 100, all_negs[i][3] * 100))

false positive: 
car, Numenta HTM, Random Cut Forest, Bayesian Changepoint, EXPoSE
1  ,  62.87%,  67.06%,  28.95%,  55.19%
3  ,  48.98%,  69.67%,  36.36%,  33.93%
4  ,  55.71%,  85.00%,  64.86%,  53.19%
6  ,  75.13%,  89.55%,  71.48%,  66.51%
7  ,  43.88%,  77.39%,  44.57%,  60.00%
12 ,  53.62%,  64.97%,  43.18%,  72.96%
14 ,  46.43%,  67.91%,  44.91%,  71.35%
15 ,  80.89%,  90.67%,  66.91%,  89.08%
17 ,  55.91%,  76.38%,  61.58%,  74.81%
18 ,  62.60%,  84.55%,  50.68%,  54.46%
19 ,  42.02%,  61.43%,  37.14%,  64.34%
20 ,  67.36%,  66.99%,  40.11%,  51.75%
21 ,  46.99%,  64.92%,  44.12%,  52.73%
22 ,  50.91%,  78.30%,  53.85%,  55.07%
23 ,  64.19%,  88.08%,  61.24%,  58.95%
24 ,  72.66%,  72.65%,  47.75%,  70.49%
25 ,  42.65%,  91.57%,  69.88%,  66.67%
26 ,  44.26%,  66.67%,  36.30%,  60.98%
27 ,  85.09%,  82.93%,  68.69%,  67.88%
28 ,  62.42%,  79.78%,  56.33%,  70.70%
29 ,  60.18%,  69.67%,  44.21%,  48.53%
59 ,  51.92%,  87.50%,  66.90%,  65.17%
60 ,  48.55%,  72.17%,  45.16%,  64.2

#### New labels for car 1 & car 13

In [428]:
new_labels = np.loadtxt('labels/indy2018-13-labels-new.txt')
new_labels[:3], new_labels.shape

(array([[60194700., 60209500.],
        [60219600., 60230700.],
        [60232100., 60273700.]]), (18, 2))

In [513]:
d_i = 3
detect = detectors[d_i]
thresh = thresholds[d_i]
csvfil = 'results/' + detect + '/anomaly_' + thresh + '_indy2018-13-vspeed.csv'
csvfil

'results/expose/anomaly_0.992_indy2018-13-vspeed.csv'

In [514]:
alldat = np.loadtxt(csvfil)
alldat[:3], alldat.shape

(array([[5.8980558e+07, 1.5169000e+02, 0.0000000e+00],
        [5.8980651e+07, 1.5290000e+02, 0.0000000e+00],
        [5.8980735e+07, 1.5378000e+02, 0.0000000e+00]]), (35441, 3))

In [515]:
mask_s = alldat[:, 2] == 1
anomal = alldat[mask_s]
anomal[:3], anomal.shape

(array([[5.8982074e+07, 1.6456000e+02, 1.0000000e+00],
        [5.8982167e+07, 1.6530000e+02, 1.0000000e+00],
        [5.8982260e+07, 1.6638000e+02, 1.0000000e+00]]), (61, 3))

In [516]:
speeds = anomal[:, 0]
speeds[:3], speeds.shape

(array([58982074., 58982167., 58982260.]), (61,))

In [529]:
tol = 5000
mask_1 = speeds >= new_labels[0, 0] - tol
mask_2 = speeds <= new_labels[-1, 1] + tol
mask_a = mask_1 & mask_2
middle = speeds[mask_a]
middle[:3], middle.shape

(array([60195644., 60197495., 60197681.]), (55,))

In [530]:
k = 0
matches = [0] * new_labels.shape[0]
for j in range(middle.shape[0]):
    if middle[j] < new_labels[k, 0] - tol:
        continue
    elif middle[j] >= new_labels[k, 0] - tol and \
            middle[j] <= new_labels[k, 1] + tol:
        matches[k] += 1
    elif middle[j] > new_labels[k, 1] + tol:
        k += 1
        j -= 1
    else:
        print('in else')
    if k == new_labels.shape[0]:
        break

matchtot = np.sum([1 for cnt in matches if cnt > 0])
missnumb = middle.shape[0] - np.sum(matches)
recallrate = matchtot / new_labels.shape[0] * 100
falsepositive = missnumb / middle.shape[0] * 100

print(csvfil)
print('recall rate: {:5.2f}%, rate of false positive: {:5.2f}%'\
      .format(recallrate, falsepositive))

results/expose/anomaly_0.992_indy2018-13-vspeed.csv
recall rate: 33.33%, rate of false positive: 30.91%
